<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/llama_index_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install llama-index

In [ ]:
%%capture
!pip install sentence-transformers

In [ ]:
import pandas as pd
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex
from llama_index.llms import HuggingFaceLLM

In [ ]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# Dataframe processing (CSV format is expected, minimal text pre-processing at the moment (write you own if you needed it :) ), change the file name as needed)
df = pd.read_csv('accnt_issues.csv')

In [ ]:
#above code wrapped in a function
def text_input(file = 'alpha_test.csv'):
  df = pd.read_csv(file)
  df_clean = df[df['text'].apply(lambda x: isinstance(x, str))]
  texts = [item.replace("\t", " ") for item in df_clean['text']]
  return texts

In [ ]:
texts = text_input('accnt_issues.csv')[:5]

In [ ]:
len(texts)

In [ ]:
#document list
documents = [Document(text=t) for t in texts]

In [ ]:
%%capture
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
#from langchain.llms import openai
import openai
#openai.api_key = ""  # Replace with your OpenAI API key
index = VectorStoreIndex.from_documents(documents)

In [ ]:
%%capture
!pip install llama_index
!pip install llama-cpp-python

In [ ]:
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin",
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

In [26]:
%%capture
!pip install transformers

In [31]:
%%capture
!pip install accelerate

In [1]:
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
import accelerate

In [ ]:
system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="StabilityAI/stablelm-tuned-alpha-3b",
    model_name="StabilityAI/stablelm-tuned-alpha-3b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
)

NameError: ignored